In [454]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

In [455]:
#count schools, students and sum budget
school_count = len(school_data_complete["school_name"].unique())
student_count = len(student_data["student_name"])
budget_sum = school_data["budget"].sum()

#convert strings to numbers
school_data_complete["math_score"] = pd.to_numeric(school_data_complete["math_score"])
school_data_complete["reading_score"] = pd.to_numeric(school_data_complete["reading_score"])

#calculate averages and total passes
math_avg = school_data_complete["math_score"].mean()
reading_avg = school_data_complete["reading_score"].mean()
math_pass = school_data_complete.loc[school_data_complete["math_score"] >= 70]
reading_pass = school_data_complete.loc[school_data_complete["reading_score"] >= 70]
both_pass = school_data_complete.loc[(school_data_complete["math_score"] >= 70) & (school_data_complete["reading_score"] >= 70)]

#get percentages
math_cent = len(math_pass) / student_count
reading_cent = len(reading_pass) / student_count
passed_both_cent = len(both_pass) / student_count

district_summary_df = pd.DataFrame(
    {"Total Schools": [school_count],
     "Total Students": [student_count],
     "Total Budget": [budget_sum],
     "Average Math Score" :[math_avg],
     "Average Reading Score":[reading_avg],
     "% Passing Math":[math_cent * 100],
     "% Passing Reading":[reading_cent * 100],
     "% Overall Passing":[passed_both_cent * 100]
     }
)

district_summary_sdf = district_summary_df.style.format({"Total Students": "{:,.0f}",
                                                        "Total Budget": "${:,.2f}"})
district_summary_sdf

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.877840,74.980853,85.805463,65.172326


## School Summary

In [456]:
#take columns we need from overall data and store in new frame
school_sum = school_data_complete[["school_name","type","size","budget","math_score", "reading_score"]]
school_sum["Per Student Budget"] = school_data_complete["budget"] / school_data_complete["size"]

#value count the number of passing students and place into data frames
mpdf = math_pass["school_name"].value_counts().to_frame().reset_index().rename(columns={"index" : "school_name", 
                                                                                 "school_name" : "math_pass"})
rpdf = reading_pass["school_name"].value_counts().to_frame().reset_index().rename(columns={"index" : "school_name", 
                                                                                 "school_name" : "reading_pass"})
bpdf = both_pass["school_name"].value_counts().to_frame().reset_index().rename(columns={"index" : "school_name", 
                                                                                 "school_name" : "both_pass"})

#left join these dataframes together
merge_df = pd.merge(school_sum, mpdf, on="school_name", how="left")
merge_df["math_pass"] = (merge_df["math_pass"] / merge_df["size"]) * 100

merge_df = pd.merge(merge_df, rpdf, on="school_name", how="left")
merge_df["reading_pass"] = (merge_df["reading_pass"] / merge_df["size"]) * 100

merge_df = pd.merge(merge_df, bpdf, on="school_name", how="left")
merge_df["both_pass"] = (merge_df["both_pass"] / merge_df["size"]) * 100

#groupby to calculate the average math and reading scores, merge this into the table
x_group = merge_df.groupby(["school_name"]).agg({"math_score": "mean", "reading_score": "mean"})
x_group = x_group.reset_index().rename(columns={"math_score":"Average Math Score","reading_score":"Average Reading Score"})
groupmerge = pd.merge(merge_df,x_group, on="school_name", how="left").drop(columns=["math_score","reading_score"])

#move the schools into the index to match example, sort alphabetically
cleaned_ssdf = groupmerge.drop_duplicates().set_index("school_name")
cleaned_ssdf.index.name = None
cleaned_ssdf = cleaned_ssdf.sort_index()

#formatting of headers
cleaned_ssdf.columns = cleaned_ssdf.columns.astype("str")
cleaned_ssdf.columns = cleaned_ssdf.columns.str.title()
cleaned_ssdf.columns = cleaned_ssdf.columns.str.replace("_", " ")
cleaned_ssdf = cleaned_ssdf.rename(columns={"Type":"School Type",
                                            "Size":"Total Students",
                                            "Budget":"Total School Budget",
                                            "Math Pass":"% Passing Math",
                                            "Reading Pass":"% Passing Reading",
                                            "Both Pass":"% Overall Passing"})

school_summary_df = cleaned_ssdf[["School Type", 
                                 "Total Students", 
                                 "Total School Budget", 
                                 "Per Student Budget", 
                                 "Average Math Score", 
                                 "Average Reading Score", 
                                 "% Passing Math", 
                                 "% Passing Reading", 
                                 "% Overall Passing"]]


school_summary_sdf = school_summary_df.style.format({"Total School Budget": "${:,.2f}",
                                                        "Per Student Budget": "${:,.2f}"})

school_summary_sdf

C:\Users\James\AppData\Local\Temp/ipykernel_4132/1674069903.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  school_sum["Per Student Budget"] = school_data_complete["budget"] / school_data_complete["size"]


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Top Performing Schools (By % Overall Passing)

In [473]:
top_5_df = school_summary_df.sort_values(by="% Overall Passing", ascending=False)
top_5_df = top_5_df.iloc[:5,:]
top_5_sdf = top_5_df.style.format({"Total School Budget": "${:,.2f}",
                                                        "Per Student Budget": "${:,.2f}"})
top_5_sdf

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Bottom Performing Schools (By % Overall Passing)

In [476]:
bot_5_df = school_summary_df.sort_values(by="% Overall Passing", ascending=True)
bot_5_df = bot_5_df.iloc[:5,:]
bot_5_sdf = bot_5_df.style.format({"Total School Budget": "${:,.2f}",
                                                        "Per Student Budget": "${:,.2f}"})
bot_5_sdf

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172
